In [68]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import json
import copy

In [69]:
st = pd.read_pickle('../IGNOBEL/Statistiche_giocatori.pkl') #importing complete stats DB
transStat = copy.deepcopy(st)
transStat.index=list(st.Id)
transStat=transStat.T # transposed df makes it easier to access players by id
#print(st.keys())
#st

In [70]:
# DB - DICTIONARY SKELETON # 

# Dictionary with specific loan info
LoanDict = {
    'start_date': '2020/08/01',
    'expire_date': '2020/07/31',
    'cost': 0,
    'redemption_right': False, #True or False
    'redemption_cost': 0, #>0 if redemption_right=True
}

# Dictionary with player current team and loan info
CurrentTeamDict = {
    'owner': '', #luca, pietro etc
    'team': '', #team_name 
    'squad': '', # 'Main' or 'Primavera'
    'start_date': '2020/08/01',
    'origin': '', #last team (e.g. team_name + ' - Primavera')
    'quotation_initial': 0,
    'on_loan': False, #True or False
    'loan_info': LoanDict
}

# Dictionary with player ownership info
ContractDict = {
    'owner': '', #this seems redundant
    'start_date': '2020/08/01',
    'cost': 0,
    'acquisition_mode': '', #asta_svincolati, draft, acquisto
    'origin': '', #team_name or svincolati
    'quotation_initial': 0
}

# Dictionary with player personal info
PersonalInfoDict = {
    'full_name': '',
    'birthdate': '1970/01/01',
    'nation': '',
    'team_real': '',
    'FC_role':''
}

# Nested dictionary for a single player info
PlayerDict = {
    'id': 0,
    'name': '',
    'personal_info': PersonalInfoDict,
    'contract': ContractDict,
    'current_team': CurrentTeamDict,
    #'transfers': TransfersDict,
    #'stats': StatsDict,
    #'seasons': SeasonStatsDict
}

# Main nested dictionary with ALL FPF players info
PlayersDB = {}

In [71]:
# IMPORT CURRENT DATA INTO DB DICTIONARY #

import copy 

for Id, Name in zip(st['Id'], st['Nome']):
    PlayersDB[Id] = copy.deepcopy(PlayerDict)
    PlayersDB[Id]['id'] = Id
    PlayersDB[Id]['name'] = Name

#print(PlayersDB)

with open('FPF_Players_Database.json', 'w') as fp:
    json.dump(PlayersDB, fp)

# Updating entire database with stats and info

Do not run this section unless something is changed in the stats database

In [72]:
def check_prim(text):
    if ' - PRIMAVERA' in text:
        out = text[:-12]
        squad = 'primavera'
    else:
        out = text
        squad = 'main'
    return out, squad

In [73]:
for Id, dic in PlayersDB.items():
    dic['personal_info']['FC_role'] = transStat[Id]['R']
    dic['personal_info']['team_real'] = transStat[Id]['Squadra']
    dic['personal_info']['full_name'] = transStat[Id]['Nome Completo']
    dic['personal_info']['nation'] = transStat[Id]['Nazionalit\'a']
    dic['personal_info']['birthdate'] = transStat[Id]['Classe']
    dic['personal_info']['age'] = transStat[Id]['Eta\'']
    dic['contract']['quotation_initial'] = transStat[Id]['Qt. I']
    dic['contract']['owner'] = transStat[Id]['Allenatore']
    dic['current_team']['owner'] = transStat[Id]['Allenatore']
    dic['current_team']['team'], dic['current_team']['squad']  = check_prim(transStat[Id]['Nome Squadra'])
    dic['current_team']['quotation_initial'] = transStat[Id]['Qt. I']
    dic['stats'] = dict(transStat[Id][4:18])

with open('FPF_Players_Database.json', 'w') as fp:
    json.dump(PlayersDB, fp)

In [74]:
# WRITE PLAYER IDS TO TXT FILE #
f = open("PlayersIds.txt", "w")
f.write('Id Name\n')
for Id,Name in zip(st['Id'], st['Nome']):
    f.write(str(Id)+' '+Name+'\n')
f.close()

# Import created database

In [75]:
with open('FPF_Players_Database.json','r') as f:
    PlayersDB_in = json.load(f)

In [78]:
#example
PlayersDB_in['2792']

{'id': 2792,
 'name': 'MUSSO',
 'personal_info': {'full_name': 'Musso Juan',
  'birthdate': '06/05/1994',
  'nation': 'Argentina, Italia',
  'team_real': 'Udinese',
  'FC_role': 'P',
  'age': '26'},
 'contract': {'owner': 'pietro',
  'start_date': '2020/08/01',
  'cost': 0,
  'acquisition_mode': '',
  'origin': '',
  'quotation_initial': 11},
 'current_team': {'owner': 'pietro',
  'team': 'PDG 1908',
  'squad': 'main',
  'start_date': '2020/08/01',
  'origin': '',
  'quotation_initial': 11,
  'on_loan': False,
  'loan_info': {'start_date': '2020/08/01',
   'expire_date': '2020/07/31',
   'cost': 0,
   'redemption_right': False,
   'redemption_cost': 0}},
 'stats': {'Pg': 4,
  'Mv': 5.88,
  'Mf': 4.38,
  'Gf': 0,
  'Gs': 6,
  'Rp': 0,
  'Rc': 0,
  'R+': 0,
  'R-': 0,
  'Ass': 0,
  'Asf': 0,
  'Amm': 0,
  'Esp': 0,
  'Au': 0}}

# In the following section we define the function that create market operations